### This notebook is used to process raw data gathered using the LinkedInProfileScraper notebook, append to existing dataframe and perform additional processing steps to produce the final result.

In [1]:
import os, random, sys, time
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import random
import re

In [57]:
df_raw = pd.read_csv('df_raw.csv') #Raw data
df_results = pd.read_csv('results.csv') #Existing data

In [58]:
df_raw.head()

,linkedin_profiles,comp_dirs,companies,org_types,titles,dates
0,https://www.linkedin.com/in/susan-wolak-8076b13/,NaN,NaN,NaN,NaN,Jan 1900 – Jan 1900
1,https://www.linkedin.com/in/luis-torres-2ba10a11/,/company/bdo-usa-llp/,"BDO USA, LLP",Accounting,Assurance Partner,Greater New York City Area
2,https://www.linkedin.com/in/luis-torres-2ba10a11/,/company/bdo-usa-llp/,"BDO USA, LLP",Accounting,Partner/Asst Regional Technical Director,Sep 1997 – Present
3,https://www.linkedin.com/in/luis-torres-2ba10a11/,/search/results/all/?keywords=Pannell%20Kerr%2...,Pannell Kerr Foster,NaN,Staff Accountant,Nov 1994 – Sep 1997
4,https://www.linkedin.com/in/billy-hampton-996240/,/company/bdo-usa-llp/,"BDO USA, LLP",Accounting,NaN,Jan 1900 – Jan 1900


In [59]:
df_results.head()

,linkedin_profiles,comp_dirs,companies,org_types,titles,end_date,start_month,start_year
0,https://www.linkedin.com/in/ricksmetanka/,/company/haskellwhite/,Haskell & White LLP,Accounting,Audit Partner-in-Charge,Present,12.0,1996.0
1,https://www.linkedin.com/in/ricksmetanka/,/company/pwc/,Coopers & Lybrand,Accounting,Audit Manager (last position),1996,1990.0,1990.0
2,https://www.linkedin.com/in/dianewittenberg/,/company/haskellwhite/,Haskell & White LLP,Accounting,Partner,Present,2002.0,2002.0
3,https://www.linkedin.com/in/dianewittenberg/,/search/results/all/?keywords=Arthur%20Andersen,Arthur Andersen,NaN,Senior Manager,2002,1985.0,1985.0
4,https://www.linkedin.com/in/patrickkross/,/company/haskellwhite/,Haskell & White,Accounting,Partner,Present,7.0,1997.0


In [21]:
# Processing raw data
df_raw = df_raw[df_raw.dates.str.contains('–')]
df_raw['dates'] = df_raw['dates'].astype(str)
df_raw[['start_date','end_date']] = df_raw['dates'].str.split('–', expand=True)
df_raw[['start_month','start_year']] = df_raw['start_date'].apply(lambda x: x.strip()).str.split(' ', expand=True)
df_raw['start_year'].fillna(df_raw['start_month'], inplace=True)
df_raw['start_month'].replace(['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'],[1,2,3,4,5,6,7,8,9,10,11,12], inplace=True)
df_raw.drop(columns=['dates','start_date'], inplace=True)

# Merge with existing results
df_results =df_results.append(df_raw,ignore_index=True)
df_results = df_results.sort_values(by=['linkedin_profiles','start_year','start_month'])
# Replace NaN by NA for strings processing
df_results.fillna('NA', inplace=True)
# Lower case companies name and title
df_results[['companies','org_types','titles']] = df_results[['companies','org_types','titles']].apply(lambda x: x.str.lower())
# Select list of non accounting entities
acct_firms = df_results[df_results.org_types.isin(['accounting','non-profit organization management']) | df_results.titles.str.contains('volunteer')].companies.unique().tolist()
non_acct_firms = df_results[~df_results.companies.isin(acct_firms)].companies.unique()
# Remove more accounting firms from non-accounting list:
terms = ['deloitte','arthur andersen','andersen','pwc','waterhouse','kpmg','bdo','peat'
        ,'marwick','ernst','p.c','p.a','coopers','lybrand',' cpa','l j soldinger'
        ,' llp','eisneramper','fasb','aicpa','financial accounting standards board'
        ,'&','vancecronin','mann judd landau','american express tax and business services'
        ,'seymour schneidman','robson rhodes','badokh','certified','heinfeld, meech and co.'
        ,'mantyla mcreynolds',', pa','public company accounting oversight board','grant thorton'
        ,'volunteer','university','college']
more_acct_firms  = [i for t in terms for i in non_acct_firms if i.find(t) != -1]
acct_firms = list(set(acct_firms + list(set(more_acct_firms))))

In [22]:
# Filter out companies belong to the list above plus partner positions
df_non_acct = df_results[~df_results.companies.isin(acct_firms)]
df_non_acct = df_non_acct[~df_non_acct.titles.str.contains('partner')]

# Remove NA dates
df_non_acct = df_non_acct[df_non_acct.start_month.ne('NA') & df_non_acct.start_year.ne('NA')]
df_non_acct[['start_month','start_year']] = df_non_acct[['start_month','start_year']].astype('int')
df_non_acct.rename(columns={'linkedin_profiles':'weblink'}, inplace=True)
df_non_acct = df_non_acct[df_non_acct.start_year.ne(1900)]

# Rename columns and remove duplications
df_non_acct.sort_values(by=['weblink','start_year','start_month'], inplace=True)
df_non_acct.drop_duplicates(inplace=True)

In [37]:
# Rank job titles based on start dates
links = df_non_acct.weblink.unique()
pos_num = []

for l in links:
    df = df_non_acct[df_non_acct.weblink.eq(l)]
    i = 1
    for n in range((non_acct_pos:=df.shape[0])):
        pos_num.append(i)
        i += 1
pos_num = np.array(pos_num)
df_non_acct['position_number'] = pos_num

In [63]:
df_non_acct.head(20)

,weblink,comp_dirs,companies,org_types,titles,end_date,start_month,start_year,position_number
4482,https://www.linkedin.com/in/abba-blum-43a4874/,/search/results/all/?keywords=MN%20Blum%20llc,mn blum llc,na,member,Present,8,2013,1
2530,https://www.linkedin.com/in/ahmad-ejaz-9714735/,/company/accounting-and-consulting-group/,accounting and consulting group,na,senior accountant,Nov 2003,3,2003,1
5034,https://www.linkedin.com/in/ahmed-ed-farag-33b...,/search/results/all/?keywords=USAID,usaid,na,internal audit manager,Nov 1999,2,1997,1
3046,https://www.linkedin.com/in/ahmed-hamdy-85b3045/,/search/results/all/?keywords=Premier%20Commer...,premier commercial brokerage,na,controller,1999,1997,1997,1
3199,https://www.linkedin.com/in/alexcastellicpa/,/search/results/all/?keywords=Retail%20Chain,retail chain,na,controller,Dec 1993,10,1989,1
3992,https://www.linkedin.com/in/alfred-fontanella-...,/search/results/all/?keywords=Fontanella%20Ass...,fontanella associates llc,na,managing member,Present,8,2015,1
4954,https://www.linkedin.com/in/alfredo-cepero-401...,/company/alpfa/,alpfa,civic & social organization,past national president,Present,2002,2002,1
2819,https://www.linkedin.com/in/alison-tjosvold-mi...,/company/pierswells/,independent consultant,"chicago, il",consultant,Oct 2020,2,2020,1
2818,https://www.linkedin.com/in/alison-tjosvold-mi...,/company/organovo-inc-/,"organovo holdings, inc.",biotechnology,director | audit committee chair | nominating ...,Present,9,2020,2
2817,https://www.linkedin.com/in/alison-tjosvold-mi...,/company/erasca/,"erasca, inc.",biotechnology,executive director,Present,10,2020,3
